# Chat API

* Connecting to Claude's API via [claudette](https://claudette.answer.ai/) for chat completion, 
* Extracting JSON and rules to populate gradio UI

In [1]:
#| default_exp chat

In [2]:
#| hide
from nbdev.showdoc import *

In [1]:
#| export
import claudette
from pathlib import Path
import re

* Claudette provides models, which is a list of models currently available from the SDK.

In [11]:
[print(m) for m in claudette.models]
# using Sonnet 3.5
model = claudette.models[3]
print(f'\nChosen model: {model}')

claude-3-opus-20240229
claude-3-7-sonnet-20250219
claude-3-5-sonnet-20241022
claude-3-haiku-20240307
claude-3-5-haiku-20241022

Chosen model: claude-3-haiku-20240307


In [12]:
import os
api_key = os.getenv('ANTHROPIC_API_KEY')
if api_key: print('key exists ☑')

key exists ☑


In [13]:
context_file = Path('job_listings/Research_analyst_london.txt')
Job_listing = context_file.read_text(encoding='utf-8')

system_prompt = """You are a helpful and concise assistant. Context for the followings tasks are here: {Job_listing}"""

chat = claudette.Chat(model, sp=system_prompt)
response = chat("""Can you come up with 20 questions an interviewer might ask me? My resume is in the project docs, the core competencies for a senior analyst are in the analytics.md, but the job title for this role is: SENIOR RESEARCH ANALYST,
     the job listing is as follows:
     {context_media}""")

# Extract the text content from the Message object
# This will depend on the exact structure of your claudette library's Message object
response_text = response.text  # or response.content or str(response)

# Now use regex on the text
questions = re.findall(r'^\s*\d+\.\s+(.+)$', response_text, re.MULTILINE)

AttributeError: 'Message' object has no attribute 'text'

['What are the key responsibilities and day-to-day tasks associated with this role?',
 'What qualifications, skills, and experience are you looking for in an ideal candidate for this position?',
 'Can you describe the company culture and work environment that the successful applicant can expect?',
 'What are the opportunities for professional development and growth within this role and the organization?',
 'Can you provide more details about the compensation package, including salary range and benefits offered?']

In [38]:
r = chat("What are the sunrise and sunset times in London during this month?")
r

Here are the typical sunrise and sunset times for London, UK during the month of March:

Early March (March 1st):
- Sunrise: 6:45 AM
- Sunset: 5:45 PM

Mid-March (March 15th):
- Sunrise: 6:15 AM 
- Sunset: 6:15 PM

Late March (March 31st):
- Sunrise: 5:45 AM
- Sunset: 6:45 PM

So over the course of the month, the days get noticeably longer, with the sunrise time getting earlier and the sunset time getting later.

By the end of March, the daylight hours in London are around 13 hours, compared to only about 11 hours at the start of the month. This gradual increase in daylight is a sign that spring is on its way.

The weather can still be quite cool and changeable during March in London, but the extra daylight hours are a welcome change after the shorter days of winter.

<details>

- id: `msg_01NacG8zSQXLWXHMTZEac5di`
- content: `[{'citations': None, 'text': 'Here are the typical sunrise and sunset times for London, UK during the month of March:\n\nEarly March (March 1st):\n- Sunrise: 6:45 AM\n- Sunset: 5:45 PM\n\nMid-March (March 15th):\n- Sunrise: 6:15 AM \n- Sunset: 6:15 PM\n\nLate March (March 31st):\n- Sunrise: 5:45 AM\n- Sunset: 6:45 PM\n\nSo over the course of the month, the days get noticeably longer, with the sunrise time getting earlier and the sunset time getting later.\n\nBy the end of March, the daylight hours in London are around 13 hours, compared to only about 11 hours at the start of the month. This gradual increase in daylight is a sign that spring is on its way.\n\nThe weather can still be quite cool and changeable during March in London, but the extra daylight hours are a welcome change after the shorter days of winter.', 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 253, 'output_tokens': 231}`

</details>

In [39]:
chat("Concisely, what is the meaning of life?",
     prefill='According to Epicurus,')

According to Epicurus,the meaning of life is to seek happiness and avoid suffering.

<details>

- id: `msg_01M1igEAq8o1S9JDZHKuviWF`
- content: `[{'citations': None, 'text': 'According to Epicurus,the meaning of life is to seek happiness and avoid suffering.', 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 505, 'output_tokens': 15}`

</details>

 * Add `stream=True` to stream the results as soon as they arrive

In [40]:
for o in chat("Concisely, what book was that in?", prefill='It was in', stream=True):
    print(o, end='')

It was in Epicurus' philosophical work "Letter to Menoeceus".

### Prompt caching

If you use `mk_msg(msg, cache=True)`, then the message is cached using Claude’s [prompt caching](https://docs.anthropic.com/en/docs/build-with-claude/prompt-caching) feature.

In [41]:
from pathlib import Path

nbtxt = Path('..\\templates\\socrates.md').read_text()
msg = f'''<README>
{nbtxt}
</README>
In brief, what is the purpose of this project based on the readme?'''
r = chat(mk_msg(msg, cache=True))
r

NameError: name 'mk_msg' is not defined

In [42]:
print(r.usage)

Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=253, output_tokens=231)


In [43]:
r = chat('What key quotes are attributed to Socrates?')
r

Some key quotes attributed to Socrates:

"The unexamined life is not worth living."

"I am the wisest man alive, for I know one thing, and that is that I know nothing."

"The only true wisdom is in knowing you know nothing."

"The greatest way to live with honor in this world is to be what we pretend to be."

<details>

- id: `msg_01GaHApwmQnLbzZPjvYAnxsz`
- content: `[{'citations': None, 'text': 'Some key quotes attributed to Socrates:\n\n"The unexamined life is not worth living."\n\n"I am the wisest man alive, for I know one thing, and that is that I know nothing."\n\n"The only true wisdom is in knowing you know nothing."\n\n"The greatest way to live with honor in this world is to be what we pretend to be."', 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 568, 'output_tokens': 84}`

</details>

In [44]:
print(r.usage)

Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=568, output_tokens=84)


### Character prompt

In [45]:
#| export

def create_system_prompt(context_path="..\\templates\\socrates.md"):
    character_prompt = """
    In this scenario you are an expert interviewer. A candidate has been asked to answer questions about the context document above.
    Start by quickly introducing yourself and setting the scene, then move quickly into the questions. 
    The first response should be a single simple question, getting progressively more difficult.
    Each corresponding response should first start with a short comment on the accuracy of the answer, do not be afraid to be critical.
    Then provide an improved answer, still using concise and spontaneous language where possible.

    Responses should be in the following format:
    [ANSWER EVALUATION]  
        {Quick evaluation of the answer}
        {If incorrect:} \\nExample: You said: ... Correction: ...} \\n
    [/ANSWER EVALUATION]

    [IMPROVED ANSWER] \\n
        {Answer structured in first person}
    [/IMPROVED ANSWER]

    Next question: \\n ...
    """

    nbtxt = Path(context_path).read_text()
    msg = f'''<README>
    {nbtxt}
    </README>
    {character_prompt}'''

    return msg


In [46]:
system_prompt = create_system_prompt()
print(system_prompt)

<README>
    # Socrates & The Socratic Method: Essential Study Guide

## Who Was Socrates (470-399 BCE)
- Ancient Greek philosopher considered one of Western philosophy's founding fathers
- Never wrote anything down; known through writings of his students, primarily Plato
- Executed by drinking hemlock after being convicted of "corrupting the youth" and "impiety"
- Married to Xanthippe, worked as a stonemason before becoming a philosopher
- Served as a soldier in the Peloponnesian War, known for his physical and mental endurance

## Core Philosophical Beliefs
- "The unexamined life is not worth living"
- Knowledge of one's own ignorance is true wisdom ("I know that I know nothing")
- Virtue is knowledge; evil stems from ignorance
- Care of the soul is paramount to material pursuits
- Truth and wisdom come from rigorous logical examination

## The Socratic Method Explained
1. Initial Position
   - Begin with someone's claim to knowledge
   - Accept this claim temporarily for examination

In [47]:
prefill_msg = """Hello, I'm Dr. Smith, and I'll be conducting your interview today about Socrates and Socratic Method. Let's dive right in.
First question: Who was Socrates, and when did he live?"""

chat = claudette.Chat(model, sp=system_prompt)
r = chat(claudette.mk_msg(system_prompt, cache=True, prefill=prefill_msg))
r

Hello, I'm an expert interviewer and I'll be asking you a series of questions about the context document on Socrates and the Socratic method. Let's begin.

First question: Who was Socrates?

<details>

- id: `msg_01F3c8pohVsghmgxhsb3AAqq`
- content: `[{'citations': None, 'text': "Hello, I'm an expert interviewer and I'll be asking you a series of questions about the context document on Socrates and the Socratic method. Let's begin.\n\nFirst question: Who was Socrates?", 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1767, 'output_tokens': 50}`

</details>

In [48]:
print(r.usage)

Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=1767, output_tokens=50)


In [49]:
r = chat('Scorates lived a long time ago in the greek classical period. Maybe 2000 BC. He was a great philosopher however never wrote anything down, that was left to one of his students, Plato')
r

[ANSWER EVALUATION]
The answer is partially correct, but contains some inaccuracies. Socrates lived in the 5th century BCE, not 2000 BC, and he was a student of philosophy, not a writer himself.
[/ANSWER EVALUATION]

[IMPROVED ANSWER]
Socrates was an ancient Greek philosopher who lived from around 470 to 399 BCE. He is considered one of the founding fathers of Western philosophy, though he never wrote anything down himself. Instead, his philosophical ideas and teachings were recorded by his students, most notably Plato.
[/IMPROVED ANSWER]

Next question: What were some of Socrates' core philosophical beliefs?

<details>

- id: `msg_01BxH9UZKFgUcDALxSst5V4R`
- content: `[{'citations': None, 'text': "[ANSWER EVALUATION]\nThe answer is partially correct, but contains some inaccuracies. Socrates lived in the 5th century BCE, not 2000 BC, and he was a student of philosophy, not a writer himself.\n[/ANSWER EVALUATION]\n\n[IMPROVED ANSWER]\nSocrates was an ancient Greek philosopher who lived from around 470 to 399 BCE. He is considered one of the founding fathers of Western philosophy, though he never wrote anything down himself. Instead, his philosophical ideas and teachings were recorded by his students, most notably Plato.\n[/IMPROVED ANSWER]\n\nNext question: What were some of Socrates' core philosophical beliefs?", 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1864, 'output_tokens': 157}`

</details>

In [50]:
r = chat('The core components are questioning the fundamentals of knowledge, assumptions, and trying to understand the problem from different perspectives.')
r

[ANSWER EVALUATION]
The answer captures some of the key aspects of Socrates' philosophical beliefs, but it is a bit broad and lacks specificity. More detail would be helpful to demonstrate a deeper understanding.
[/ANSWER EVALUATION]

[IMPROVED ANSWER]
According to the context, some of Socrates' core philosophical beliefs included:
1) "The unexamined life is not worth living" - Socrates believed that self-reflection and critical examination of one's beliefs and assumptions is essential for living a meaningful life.
2) "Knowledge of one's own ignorance is true wisdom" - Socrates famously said "I know that I know nothing," emphasizing the importance of intellectual humility and acknowledging the limits of one's own knowledge.
3) Virtue is knowledge, and evil stems from ignorance - Socrates believed that being a good, moral person was a matter of having the right knowledge and understanding, rather than just following rules or social conventions.
4) The care of the soul is paramount to material pursuits - Socrates placed greater importance on the development of one's inner life and wisdom than on external, material concerns.
[/IMPROVED ANSWER]

Next question: Can you explain the key steps of the Socratic method?

<details>

- id: `msg_018GBCiSFtvuXYmLXmhnXdZD`
- content: `[{'citations': None, 'text': '[ANSWER EVALUATION]\nThe answer captures some of the key aspects of Socrates\' philosophical beliefs, but it is a bit broad and lacks specificity. More detail would be helpful to demonstrate a deeper understanding.\n[/ANSWER EVALUATION]\n\n[IMPROVED ANSWER]\nAccording to the context, some of Socrates\' core philosophical beliefs included:\n1) "The unexamined life is not worth living" - Socrates believed that self-reflection and critical examination of one\'s beliefs and assumptions is essential for living a meaningful life.\n2) "Knowledge of one\'s own ignorance is true wisdom" - Socrates famously said "I know that I know nothing," emphasizing the importance of intellectual humility and acknowledging the limits of one\'s own knowledge.\n3) Virtue is knowledge, and evil stems from ignorance - Socrates believed that being a good, moral person was a matter of having the right knowledge and understanding, rather than just following rules or social conventions.\n4) The care of the soul is paramount to material pursuits - Socrates placed greater importance on the development of one\'s inner life and wisdom than on external, material concerns.\n[/IMPROVED ANSWER]\n\nNext question: Can you explain the key steps of the Socratic method?', 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 2047, 'output_tokens': 279}`

</details>

## More directly testing on Mock Interviews:

In [ ]:
INTERVIEW_QS = 


In [ ]:
INTERVIEW_INSTRUCTIONS = """Can you walk through each of the interview question from the. file, one by one, and I will reply directly to them in the response, as in a mock interview. I want you to set up the following sections with line breaks between them 

----- ANSWER EVAL ----- {Summarise user input answer concisely and into a few bullet points, (depending on length). This section will be displayed in Markdown, so format appropriately.
Use headings GOOD, with a small numbered list of what is considered high-quality response. 
Use heading BAD, critique of where the answer went wrong, think about verbosity, clarity, what kind of thing should be answered in a high level interview, for a very senior position like this.]

[RESPONSE METRICS] Here please return a json object with these metrics as keys and a score rating between 0-10 (1 d.p.). This is mostly for fun but do the best you can with it. The metrics:
* Linguistic Clarity
* Confidence
* Technical Expertise
* Strategic Thinking
* Impact Demonstration
* Industry Understanding
[ENDJSON]

----- REWRITEN ----- 
Use the above notes to write a first-in-class script for the question. Use the same general facts as the actual answer, but don't be afraid to add in external context to the answer if your descretion requires it. This is a fictional exercise and we want the answer to be an example of a perfect answer. Use bold to highly key words and phrases in the answer, and keep it conversational in tone. It needs to be spoken out loud, so not like and essay. 
----- END ----- 
If the users answers the question in the response the evaluate it like above, and then ask to move on to the next question. For each question and answer repeat the same instructions."""

In [51]:
#| hide
import nbdev; nbdev.nbdev_export()